In [170]:
import pandas as pd
from ast import literal_eval
import json
import sys


Cargando los dataframes

In [171]:
reviews_rows = []
with open("datasets/australian_user_reviews.json") as f:
    for line in f.readlines():
        reviews_rows.append(literal_eval(line))

df_reviews = pd.DataFrame(reviews_rows)


In [172]:
usersdata_rows = []
with open("datasets/australian_users_items.json") as f:
    for line in f.readlines():
        usersdata_rows.append(literal_eval(line))

df_usersdata = pd.DataFrame(usersdata_rows)

In [173]:
games_rows = []
with open("datasets/output_steam_games.json") as f:
    for line in f:
        games_rows.append(json.loads(line))

df_games = pd.DataFrame(games_rows)

-------------------------------------------------------------------------------------

#### DF_REVIEWS

Info general:

Cambios generales antes de pasar a las modificaciones para las funciones:
        
        - Desanidar reviews


Columnas:

        user_id : Id del usuario

        user_url: link steam profile

        reviews: Un JSON anidado con cada datos de cada reseña hecha, con los campos:
                "funny":"" ( todos estan vacios)
                "posted": fecha que se escribio la reseña
                "item_id": ID del juego 
                "helpful": Si la recibió apoyo por otros users o no
                "review": La review del usuario.

In [174]:
# Revisión de nulos
df_reviews.isna().sum()



user_id     0
user_url    0
reviews     0
dtype: int64

In [175]:
# Verificando duplicados
print(f'Valores duplicados en user_id: {df_reviews["user_id"].duplicated().sum()}')
print(f'Valores duplicados en user_url: {df_reviews["user_url"].duplicated().sum()}')

Valores duplicados en user_id: 314
Valores duplicados en user_url: 314


------------------------------------------------------------------------------------------------------

#### DF_USERSDATA

Preview general de los datos originales

In [176]:
df_usersdata.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


Columnas:

        "user_id": nombre del usuario

        "items_count": cantidad de items del usuario

        "steam_id": id de steam

        "user_url": link al perfil de steam

        "itemss": Un JSON anidado con datos de cada juego de la cuenta, con los campos:
                        "item_id": id del juego
                        "item_name": nombre del juego
                        "playtime_forever": Tiempo total jugado
                        "playtime_2weeks": Tiempo jugado en las últimas dos semanas


Cambios generales antes de pasar a las modificaciones para las funciones:
        
        - Cambiar la columna "items" a "itemss" para que no se confunda con una
        funcion de pandas
        - Comprobar duplicados entre USER_ID, STEAM_ID Y USER_URL
        - Borrar duplicados del que menos tenga
        - Borrar las dos columnas que tuvieron más duplicados
        - Crear un dataframe con la columna itemss desanidada 
        - Eliminar las columnas que no estaban anidadas en el dataframe original
        
        - Eliminar la columna ITEMSS en el dataframe original
        - 
        - Unirlos para las querys a través de la relación uno a muchos como en sql

Cambios generales antes de pasar a las modificaciones para las funciones:

        - Cambiar la columna "items" a "itemss" para que no se confunda con una
        funcion de pandas

In [177]:

df_usersdata.rename(columns={"items":"itemss"},inplace=True)

        - Comprobar duplicados entre USER_ID, STEAM_ID Y USER_URL

In [178]:
print(f'Duplicados en steam_id :{df_usersdata["steam_id"].duplicated().sum()}')
print(f'Duplicados en user_id: {df_usersdata["user_id"].duplicated().sum()}')
print(f'Duplicados en steam_url :{df_usersdata["user_url"].duplicated().sum()}')

# Conlusión:
# Nos quedamos con user id ya que es el que menos repetidos tiene, y usar user_url para 
#relacionar tablas no tiene sentido


Duplicados en steam_id :685
Duplicados en user_id: 684
Duplicados en steam_url :684


        - Borrar duplicados del que menos tenga


In [179]:
df_usersdata.drop_duplicates(subset=["user_id"],inplace=True)


        - Desanidar ITEMSS

In [180]:
data_desanidada = []

for index, row in df_usersdata.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    items = row['itemss']
    
    for i in items:   
        new_row = {
        'user_id': user_id,
        'items_count': items_count,
        'steam_id' : steam_id,
        'user_url' : user_url,
        'item_id': i.get('item_id', ''),
        'item_name': i.get('item_name', ''),
        'playtime_forever': i.get('playtime_forever', ''),
        'playtime_2weeks': i.get('playtime_2weeks', '')
        }
        
        data_desanidada.append(new_row)

df_usersdata_desanidada = pd.DataFrame(data_desanidada)

        - Borrar las dos columnas que tuvieron más duplicados

In [181]:
df_usersdata.drop(columns=["steam_id","user_url"],inplace=True)

In [182]:
print(f"n° de registros y columnas original: {df_usersdata.shape}")
print(f"n° de registros y columnas desanidado: {df_usersdata_desanidada.shape}")

n° de registros y columnas original: (87626, 3)
n° de registros y columnas desanidado: (5094082, 8)


In [183]:
df_usersdata_desanidada.head(1)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0


        - Crear un dataframe nuevo sólo con las columnas que estaban anidadas en ITEMSS y con USER_ID para relacionar los dataframe
        

In [184]:

columnas_deseadas =["user_id","item_id","item_name","playtime_forever","playtime_2weeks"]

df_usersdata_only_items = df_usersdata_desanidada[columnas_deseadas].copy()

In [185]:
df_usersdata_only_items.head(2)

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0


        - Borrar duplicados del dataframe de los items

In [186]:
df_usersdata_only_items["item_name"].duplicated().sum()
df_usersdata_only_items

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...
5094077,76561198329548331,346330,BrainBread 2,0,0
5094078,76561198329548331,373330,All Is Dust,0,0
5094079,76561198329548331,388490,One Way To Die: Steam Edition,3,3
5094080,76561198329548331,521570,You Have 10 Seconds 2,4,4


        - Eliminar la columna ITEMSS en el dataframe original

In [187]:
df_usersdata.drop(columns=["itemss"], inplace=True)

        - Eliminar el dataframe que fue usado para dividir las tablas (hay que cuidar esa ram que esto se va a hacer deploy en un host gratuito xD )

In [188]:
# Gracias por todo compañero 
df_usersdata_desanidada = None

In [189]:
df_usersdata_only_items.head(3)

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0


In [190]:
df_usersdata.head(3)

,user_id,items_count
0,76561197970982479,277
1,js41637,888
2,evcentric,137


------------------------------------------------------------------------------------------------

#### DF_GAMES

Info general:

In [191]:

df_games_tamanio_original = sys.getsizeof(df_games) / 1e+6





Cantidad de registros : 120445

Columnas:    
                                                       
        Columnas                                                        Cantidad de nulos

        "publisher": nombre del publisher                              | 96362
        
        "genres": lista con los generos                                | 91593

        "app_name": nombre de la app                                   | 88312

        "tittle": titulo de la app                                     | 90360

        "url": link a la steampage del juego                           | 88310

        "release_date": fecha de salida del juego                      | 90377

        "tags": como "genres" pero parece que con mas items en la lista| 88473

        "reviews_url": link a las reviews                              | 88312

        "specs": lista con especificaciones del juego                  | 88980

        "price": precio                                                | 89687

        "early_access": booleano                                       | 88310

        "id": id del juego                                             | 88312

        "developer": nombre del desarrollador                          | 91609


Taks generales antes de pasar a las modificaciones para las funciones:
        
        1. Borrar las columnas que no se usen en ninguna funcion:
        "publisher", "url", "reviews_url", "price", "early_access", "developer"

        2. Tratamiento de nulos:
        
                2.a. Empezar eliminando los que no tengan ningun dato

                2.b. Eliminar los que no tengan release_date (columna clave)

        3. Cambiar la fecha de a formato datetime
        4.a Comprobar TITTLE y APP_NAME tienen el mismo dato
        4.b Comprobar duplicados entre TITTLE y APP_NAME
        4.c Quedarse con el que menos duplicados tenga

        

In [192]:
# Cantidad de registros iniciales
df_games.shape[0]

120445

In [193]:
# task 1

df_games.drop(columns=["publisher", "url", "reviews_url", "price", "early_access", "developer"], inplace= True)

In [194]:
# task 2.a

df_games.dropna(how="all",inplace=True)



In [195]:
# task 2.b

df_games.dropna(subset=["release_date"],inplace=True)

In [196]:
# task 3

df_games["release_date"] = pd.to_datetime(df_games["release_date"],format="%Y-%m-%d",errors="coerce")

# eliminamos las fechas corruptas
df_games.dropna(subset=["release_date"],inplace=True)

In [197]:
# task 4.a

comparacion = df_games["title"] == df_games["app_name"]

print(f"Cantidad de registros: {df_games.shape[0]}\nCantidad de datos identicos: {comparacion.sum()}")

# Con cúal me quedo?  
# con app_name


Cantidad de registros: 29825
Cantidad de datos identicos: 29270


In [198]:
# task 4.b

print(f"N° de duplicados en {df_games['app_name'].duplicated().sum()}")
print(f"N° de duplicados en {df_games['title'].duplicated().sum()}")

# conclusion: Elimino cualquiera de los dos que ya tienen los m

N° de duplicados en 30
N° de duplicados en 30


In [200]:
#task 4.c borrar tittle

df_games.drop(columns=["title"],inplace=True)

In [201]:
# Nulos finales
df_games.isna().sum()

genres          1234
app_name           1
release_date       0
tags             161
specs            669
id                 1
dtype: int64

In [202]:
# Tamaño en memoria post cambios

df_games_tamanio_postcambios = sys.getsizeof(df_games) / 1e+6

print(f"Espacio en memoria original: {str(df_games_tamanio_original)[0:5]} MB")
print(f"Espacio en memoria del dataset postcambios: {str(df_games_tamanio_postcambios)[0:5]} MB")




Espacio en memoria original: 70.26 MB
Espacio en memoria del dataset postcambios: 13.50 MB


In [203]:
df_games.head(2)

,genres,app_name,release_date,tags,specs,id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],761140
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",643980


In [215]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29825 entries, 88310 to 120443
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28591 non-null  object        
 1   app_name      29824 non-null  object        
 2   release_date  29825 non-null  datetime64[ns]
 3   tags          29664 non-null  object        
 4   specs         29156 non-null  object        
 5   id            29824 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 1.6+ MB


-------------------------

#### Vistazo general de los principales dataframes

Reviews de los usuarios

In [204]:
df_reviews.head(2)


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."


Datos del perfil de los usuario y sus items 

In [205]:
df_usersdata.head(2)

,user_id,items_count
0,76561197970982479,277
1,js41637,888


In [206]:
df_usersdata_only_items.head(2)

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0


Info de los juegos

In [207]:
df_games.head(2)

,genres,app_name,release_date,tags,specs,id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],761140
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",643980


In [220]:
df_games.columns

Index(['genres', 'app_name', 'release_date', 'tags', 'specs', 'id'], dtype='object')

#### Creando un dataframe para cada función

##### df_playtimegenre

        - Borrar el dato nulo que tiene app_name
        POR ALGUN MOTIVO QUE DESCONOZCO NO SE BORRA

In [211]:
df_games["app_name"].dropna(inplace=True)

In [213]:
df_games["app_name"].isna().sum()

1

In [214]:
filtro_dato_nulo = df_games["app_name"].isnull()
dato_nulo = df_games[filtro_dato_nulo]
print(dato_nulo)

                genres app_name release_date             tags  \
90890  [Action, Indie]      NaN   2014-08-26  [Action, Indie]   

                            specs      id  
90890  [Single-player, Game demo]  317160  


In [223]:
df_games["id"].dropna(inplace=True)

df_usersdata_only_items["item_id"].dropna(inplace=True)


df_games["id"] = df_games["id"].astype("float64")

df_usersdata_only_items["item_id"] = df_usersdata_only_items["item_id"].astype("float64")

In [228]:


columnas_dfgames_para_dfplaytimegenre = ["id","genres","app_name","release_date"]
df_games_para_playtimegenre = df_games[columnas_dfgames_para_dfplaytimegenre].copy()

columnas_dfuserdata_para_dfplaytime = ["user_id","item_id","item_name","playtime_forever",]
df_userdata_onlyitems_para_playtimegenre = df_usersdata_only_items[columnas_dfuserdata_para_dfplaytime].copy()

In [246]:
df_playtimegenre = pd.merge(df_userdata_onlyitems_para_playtimegenre,df_games_para_playtimegenre, left_on="item_id",right_on="id",how="inner")

In [247]:
df_playtimegenre.head(1)

,user_id,item_id,item_name,playtime_forever,id,genres,app_name,release_date
0,76561197970982479,10.0,Counter-Strike,6,10.0,[Action],Counter-Strike,2000-11-01


In [243]:
#df_playtimegenre["genres"] = df_playtimegenre["genres"].apply(lambda x: [x] if isinstance(x,str) else x)

In [248]:
df_playtimegenre['genres'] = df_playtimegenre['genres'].apply(lambda x: [x] if not isinstance(x, list) else x)

In [259]:
def anio_lanzamiento_mas_horas(df_ptg,genero: str):

    

    df_filtrado = df_ptg[df_ptg["genres"].apply(lambda x: genero in x)]

    grupo = df_filtrado.groupby("release_date")["playtime_forever"].sum().reset_index()

    res = grupo.loc[grupo["playtime_forever"].idxmax()]

    horas = str(res.playtime_forever)

    return horas

genero_buscar = "Action"

resultado = anio_lanzamiento_mas_horas(df_playtimegenre,genero_buscar)

print({f"Año de lanzamiento con más horas jugadas para Género {genero_buscar}": genero_buscar})
 
# {f"Año de lanzamiento con más horas jugadas para Género {genero}": maxima_horas_por_año}

#{"Año de lanzamiento con más horas jugadas para Género X" : 2013}



{'Año de lanzamiento con más horas jugadas para Género Action': 'Action'}


{'Año de lanzamiento con más horas jugadas para Género Action': 775875211}





Qué columnas y de qué dataframes tendría que usar para crear este?

                columnas:

                df_games.genre: Todos los generos

                df_games.app_name: nombre del juego


Datos de la funcion:

Recibe como parametro el género

Debe retornar el año con mas horas jugadas para dicho género.

Ejemplo de retorno: 

        {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

Cúal fue el genero que más se jugó este año?

tengo que tener el genero
el año 
la cantidad de horas totales de los jugadores 




                


        


Como tendria que ser el dataframe para esta función?

tener una columna con cada genero, 
        

----------------------------------------------

ELEGIR UNO DE DOS 

Si es un sistema de recomendación item-item:

    def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

Si es un sistema de recomendación user-item:

    def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.


Me confié porque ya habia aprobado el proyecto indivial uno en otra cohorte y le dedique menos tiempo del que deberia haberle dedicado, espero que se note la intencion que tenia de hacer todo de forma muy prolija y ordenada. voy a dedicarle todo mi esfuerzo desde el dia 1 al segundo proyecto para demostrar de lo que soy capaz. De los errores se aprende y sin duda hoy aprendí un montón.